In [1]:
import torch

print(torch.__version__)

ModuleNotFoundError: No module named 'torch'

In [ ]:
import glob as glob

pngPath = '../ue/onnx/Content/Data'

pngList = glob(pngPath + '/*.png')


In [ ]:
from ultralytics import YOLO
import cv2

# Create a YOLOv8 model
model = YOLO("yolov8n.pt")
